In [1]:
# Importing the requisite libraries
import logging
import os
import pathlib
import subprocess
import warnings
from glob import glob

import earthpy.appeears as etapp
import folium
import geopandas as gpd
import hvplot.pandas
import hvplot.xarray
import pandas as pd
import requests
import rioxarray as rxr
import xarray as xr

# Set up logging so AppeearsDownloader will log in notebook
logging.basicConfig(level=logging.INFO)

# Ignore FutureWarning coming from hvplot
warnings.simplefilter(action='ignore', category=FutureWarning)

ModuleNotFoundError: No module named 'earthpy.appeears'

In [ ]:
# Creating path to the project directory
project_dir = os.path.join(
    pathlib.Path.home(), 'earth-analytics', 'data', 'fourmile-fire')

# Creating the project directory
os.makedirs(project_dir, exist_ok=True)
project_dir

In [ ]:
# Extract GeoDataFrame from the web link
fourmile_gdf = gpd.read_file("https://services3.arcgis.com/T4QMspbfLg3qTGWY/"
                         "arcgis/rest/services/Historic_Geomac_Perimeters"
                         "_2018/FeatureServer/0/query?where=incidentname"
                         "%20%3D%20'FOURMILE%20CANYON%20FIRE'%20AND%20&"
                         "outFields=*&outSR=4326&f=json")
fourmile_gdf

In [ ]:
# Loading the area base map
fourmile_base_map = folium.Map(location=[40.0583331,-105.407165038], zoom_start=10)

# Adding location marker for Paradise, CA
folium.Marker(
    [40.0583331,-105.407165038],
    tooltip="Gold Hill, CO"
).add_to(fourmile_base_map)

# Loading fire perimeter
folium.GeoJson(
    camp_gdf,
    name='fire_perimeter_layer'
).add_to(fourmile_base_map)

fourmile_base_map

In [ ]:
# Initialize AppeearsDownloader for MODIS NDVI data
ndvi_downloader = etapp.AppeearsDownloader(
    download_key='modis-ndvi',
    ea_dir=project_dir,
    product='MYD13Q1.061',
    layer='_250m_16_days_NDVI',
    start_date='06-01',
    end_date='08-31',
    recurring=True,
    year_range=[2017, 2022],
    polygon=camp_gdf
)

# Download files if the download directory does not exist
if not os.path.exists(ndvi_downloader.data_dir):
    ndvi_downloader.download_files()

ndvi_downloader

In [ ]:
# Loading .tif files in python
ndvi_path_list = sorted(glob(os.path.join(
    ndvi_downloader.data_dir, '*', '*NDVI*.tif'))
)

ndvi_path_list

In [ ]:
doy_start = -19
doy_end = -12
scale_factor = 10000

ndvi_das = []
for ndvi_path in ndvi_path_list:
    # Get date from file name
    doy = ndvi_path[doy_start:doy_end]
    date = pd.to_datetime(doy, format='%Y%j')

    # Open dataset
    da = rxr.open_rasterio(ndvi_path, masked=True).squeeze()

    # Prepare to concatenate: Add date dimension and clean up metadata
    da = da.assign_coords({'date': date})
    da = da.expand_dims({'date': 1})
    da.name = 'NDVI'

    # Divide by scale factor
    da = da / scale_factor

    # Add the DataArray to the end of the accumulator list
    ndvi_das.append(da)

ndvi_das

In [ ]:
# Creating Time Series by date
ndvi_ds = xr.combine_by_coords(ndvi_das, coords=['date'])
ndvi_ds

In [ ]:
import matplotlib.pyplot as plt
# Plotting change in NDVI over time
dndvi_da = (ndvi_ds
        .sel(date='2019',)
        .mean('date')
        .NDVI
    - ndvi_ds
        .sel(date='2017',)
        .mean('date')
        .NDVI
)

dndvi_da.plot(cmap=plt.colormaps['PiYG'])
camp_gdf.plot(facecolor='none',  ax=plt.gca())

plt.show()

In [ ]:
# Compare the area in/out of fire boundary
out_gdf = (
    gpd.GeoDataFrame(geometry=camp_gdf.envelope)
    .overlay(camp_gdf, how="difference")
)

out_gdf

In [ ]:
# Find the yearly mean NDVI inside and outside fire boundary
inside_df = ndvi_fire_da.groupby('date.year').mean(...).NDVI.to_dataframe()

outside_df = ndvi_out_da.groupby('date.year').mean(...).NDVI.to_dataframe()

inside_df, outside_df

In [ ]:
# Finding the difference in NVDI in/out of fire boundary over time
ndvi_diff_df = outside_df.NDVI - inside_df.NDVI
ndvi_diff_df

# Plot showing NDVI Difference
ndvi_diff_df.plot(
    xlabel='year', ylabel='Difference in NDVI'
)